In [34]:
import plotly.graph_objects as go
import json
import numpy as np

In [35]:
# Constants
img_width = 1030
img_height = 960
scale_factor = 1

np.random.seed(1)

with open('sensors_list.json', 'r') as f:
    sensors = json.load(f)
    
sensors_len = len(sensors)
sensors_ids = list(map(lambda x: x['id'], sensors))

with open('measurements/outer_sensors_per_sensor_importance.json', 'r') as f:
    importance = json.load(f)

inner_ids = list(importance.keys())
outer_ids = list(importance[inner_ids[0]].keys())

importance_matrix = [[0 for _ in range(sensors_len)] for _ in range(sensors_len)]

for inner_id in inner_ids:
    matrix_inner_id = 0
    inner_dict = importance[inner_id]
    
importance_dict = {}
for sensor_id in sensors_ids:
    if str(sensor_id) in inner_ids:
        importance_array_inner = []
        for sensor_id_inner in sensors_ids:
            if str(sensor_id_inner) in outer_ids:
                importance_array_inner.append(importance[str(sensor_id)][str(sensor_id_inner)])
            else:
                importance_array_inner.append(0.002)
        importance_dict[sensor_id] = importance_array_inner
    else:
        importance_array_inner = []
        for sensor_id_inner in sensors_ids:
            if str(sensor_id_inner) in outer_ids:
                importance_array_inner.append(0.005)
            else:
                importance_array_inner.append(0.002)
        importance_dict[sensor_id] = importance_array_inner

x = list(map(lambda s: s['location']['longitude'], sensors))
y = list(map(lambda s: s['location']['latitude'], sensors))

f = go.FigureWidget([go.Scatter(x=x, y=y, mode='markers')])

# Configure axes
f.update_xaxes(
    visible=True,
    range=[19.70, 20.20], #[0, img_width * scale_factor]
)

f.update_yaxes(
    visible=True,
    range=[49.90, 50.20], #[0, img_height * scale_factor],
    # the scaleanchor attribute ensures that the aspect ratio stays constant
    #scaleanchor='x'
)

# Add image
f.add_layout_image(
    dict(
        x=19.70,
        sizex=0.5,
        y=50.20,
        sizey=0.3,
        xref='x',
        yref='y',
        opacity=0.5,
        layer='below',
        sizing='stretch',
        source='charts/cracow3.png')
)

# Configure other layout
f.update_layout(
    width=img_width * scale_factor,
    height=img_height * scale_factor,
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
)

scatter = f.data[0]
colors = np.random.rand(len(x))
scatter.marker.color = colors
scatter.marker.showscale = True
scatter.marker.size = [10] * len(x)
f.layout.hovermode = 'closest'


# create our callback function
def update_point(trace, points, selector):
    i = points.point_inds[0]
    c = importance_dict[sensors_ids[i]]
    s = [10] * len(x)
    with f.batch_update():
        scatter.marker.color = c
        scatter.marker.size = list(map(lambda x: x * 5000, c))


scatter.on_click(update_point)

f

FigureWidget({
    'data': [{'marker': {'color': array([4.17022005e-01, 7.20324493e-01, 1.14374817e-04, 3.0233…